In [ ]:
import os
import numpy as np
import pandas as pd
import mne
from mne.decoding import CSP
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, 
                                    DepthwiseConv2D, AvgPool2D, Dropout, Flatten, Dense)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ReduceLROnPlateau, EarlyStopping, 
                                       ModelCheckpoint)
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# =============================================================================
# Configuration
# =============================================================================
MTC_BASE_PATH = '/kaggle/input/mtcaic3'
EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
MOTION_THRESHOLD = 7.7966
FINAL_Z_SCORE_THRESHOLD = 5.0
MI_SAMPLES = 2250  # 9s * 250Hz
SSVEP_SAMPLES = 1750  # 7s * 250Hz
SFREQ = 250  # Sampling frequency
CSP_COMPONENTS = 4  # Number of CSP components to use

# EEGNet Hyperparameters
DROPOUT_RATE = 0.5
KERNEL_LENGTH = 32
F1 = 8
D = 2
F2 = 16
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-3

# =============================================================================
# Helper Functions
# =============================================================================
def load_trial_data(row, base_path):
    """Load raw EEG data for a specific trial"""
    id_num = row['id']
    if id_num <= 4800:
        split = 'train'
    elif id_num <= 4900:
        split = 'validation'
    else:
        split = 'test'
    
    task = row['task']
    eeg_path = os.path.join(
        base_path, task, split, row['subject_id'], 
        str(row['trial_session']), 'EEGdata.csv'
    )
    
    if not os.path.exists(eeg_path):
        return None
    
    eeg_data = pd.read_csv(eeg_path)
    trial_num = int(row['trial'])
    
    if task == 'MI':
        samples_per_trial = MI_SAMPLES
    else:  # SSVEP
        samples_per_trial = SSVEP_SAMPLES
        
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    return eeg_data.iloc[start_idx:end_idx]

def check_motion_artifact(trial_df, threshold):
    """Check for motion artifacts in trial data"""
    motion_sensors = ['AccX', 'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
    if trial_df[motion_sensors].std().max() > threshold:
        return True
    return False

def check_zscore_artifact(trial_df, z_threshold):
    """Check for signal artifacts using Z-scores"""
    for channel in EEG_CHANNELS:
        if channel in trial_df.columns:
            data = trial_df[channel].values
            mean = np.mean(data)
            std = np.std(data)
            if std < 1e-8: 
                continue
            z_scores = (data - mean) / std
            if np.any(np.abs(z_scores) > z_threshold):
                return True
    return False

def apply_ica(subject_df):
    """Apply ICA for artifact removal"""
    info = mne.create_info(
        ch_names=EEG_CHANNELS, 
        sfreq=SFREQ, 
        ch_types='eeg'
    )
    # Convert to volts (µV to V)
    raw = mne.io.RawArray(
        subject_df[EEG_CHANNELS].T.values * 1e-6, 
        info, 
        verbose=False
    )
    
    # Bandpass filter before ICA
    raw.filter(l_freq=1.0, h_freq=40.0, fir_design='firwin', verbose=False)
    
    # Apply ICA
    ica = mne.preprocessing.ICA(
        n_components=len(EEG_CHANNELS),
        random_state=97,
        max_iter='auto',
        verbose=False
    )
    ica.fit(raw)
    
    # Remove EOG artifacts
    eog_indices, _ = ica.find_bads_eog(raw, ch_name='FZ')
    ica.exclude.extend(eog_indices)
    ica.apply(raw, verbose=False)
    
    # Return cleaned data in µV
    return raw.to_data_frame().values[:, :len(EEG_CHANNELS)] * 1e6

def bandpass_filter(data, l_freq, h_freq, sfreq=SFREQ):
    """Apply zero-phase bandpass filter to EEG data"""
    info = mne.create_info(
        ch_names=[f'CH{i}' for i in range(data.shape[1])], 
        sfreq=sfreq, 
        ch_types='eeg'
    )
    raw = mne.io.RawArray(data.T, info, verbose=False)
    raw.filter(l_freq=l_freq, h_freq=h_freq, method='fir', 
              fir_design='firwin', verbose=False)
    return raw.get_data().T

def EEGNet(nb_classes, Chans=64, Samples=128, 
           dropoutRate=DROPOUT_RATE, kernLength=KERNEL_LENGTH, 
           F1=F1, D=D, F2=F2, dropoutType='Dropout'):
    """Keras Implementation of EEGNet"""
    if dropoutType == 'SpatialDropout2D':
        dropoutType = tf.keras.layers.SpatialDropout2D
    else:
        dropoutType = Dropout

    input1 = Input(shape=(Chans, Samples, 1))
    block1 = Conv2D(F1, (1, kernLength), padding='same', 
                   use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), depth_multiplier=D,
                            depthwise_constraint=tf.keras.constraints.max_norm(1.),
                            use_bias=False)(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AvgPool2D((1, 4))(block1)
    block1 = dropoutType(dropoutRate)(block1)
    
    block2 = Conv2D(F2, (1, int(kernLength//4)), padding='same',
                   use_bias=False)(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AvgPool2D((1, 8))(block2)
    block2 = dropoutType(dropoutRate)(block2)
    
    flatten = Flatten(name='flatten')(block2)
    dense = Dense(nb_classes, name='dense', 
                 kernel_constraint=tf.keras.constraints.max_norm(0.25))(flatten)
    softmax = Activation('softmax', name='softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

# =============================================================================
# Main Processing Pipeline
# =============================================================================
# Load all data indices
train_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'train.csv'))
val_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'validation.csv'))
test_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'test.csv'))

# Filter for MI tasks only
mi_train = train_df[train_df['task'] == 'MI'].copy()
mi_val = val_df[val_df['task'] == 'MI'].copy()
mi_test = test_df[test_df['task'] == 'MI'].copy()

# Store preprocessed data
preprocessed = {'train': [], 'val': [], 'test': []}
labels = {'train': [], 'val': []}
ids = {'train': [], 'val': [], 'test': []}

# =============================================================================
# Step 1: Preprocessing Pipeline
# =============================================================================
print("Starting preprocessing pipeline...")
for df, split in zip([mi_train, mi_val, mi_test], ['train', 'val', 'test']):
    print(f"\nProcessing {split} set ({len(df)} trials)")
    subjects = df['subject_id'].unique()
    subject_data = {s: [] for s in subjects}
    
    # Group trials by subject
    for _, row in df.iterrows():
        trial_data = load_trial_data(row, MTC_BASE_PATH)
        if trial_data is None:
            continue
            
        # Step 1a: Motion artifact detection
        if check_motion_artifact(trial_data, MOTION_THRESHOLD):
            continue
            
        # Store raw data for subject-wise processing
        subject_data[row['subject_id']].append((row, trial_data))
    
    # Process each subject's data
    for subject, trials in subject_data.items():
        if not trials:
            continue
            
        # Combine all trials for the subject
        all_trials = [t[1] for t in trials]
        subject_df = pd.concat(all_trials, ignore_index=True)
        
        # Step 1b: Center data
        subject_means = subject_df[EEG_CHANNELS].mean()
        centered_df = subject_df[EEG_CHANNELS] - subject_means
        
        # Step 1c: Apply ICA
        try:
            ica_cleaned = apply_ica(centered_df)
        except:
            continue
            
        # Step 1d: Split back into trials and process individually
        for i, (row, _) in enumerate(trials):
            start_idx = i * MI_SAMPLES
            end_idx = start_idx + MI_SAMPLES
            trial_cleaned = ica_cleaned[start_idx:end_idx]
            
            # Step 1e: Z-score artifact rejection
            if check_zscore_artifact(
                pd.DataFrame(trial_cleaned, columns=EEG_CHANNELS),
                FINAL_Z_SCORE_THRESHOLD
            ):
                continue
                
            # Step 1f: Trial-wise standardization
            trial_std = (trial_cleaned - trial_cleaned.mean(axis=0)) 
            trial_std /= trial_std.std(axis=0)
            
            # Store preprocessed data
            preprocessed[split].append(trial_std)
            if split in ['train', 'val']:
                labels[split].append(row['label'])
            ids[split].append(row['id'])
            
    print(f"  Kept {len(preprocessed[split])} trials after preprocessing")

# =============================================================================
# Step 2: Bandpass Filtering & CSP
# =============================================================================
print("\nApplying bandpass filtering (8-30 Hz) and CSP...")

# Bandpass filter all trials
for split in ['train', 'val', 'test']:
    for i, trial in enumerate(preprocessed[split]):
        preprocessed[split][i] = bandpass_filter(
            trial, l_freq=8, h_freq=30, sfreq=SFREQ
        )

# Prepare data for CSP
X_train = np.array(preprocessed['train'])
y_train = np.array(labels['train'])

# Encode labels to integers
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Fit CSP on training data
csp = CSP(n_components=CSP_COMPONENTS, reg=None, log=True, norm_trace=False)
csp.fit(X_train, y_train_encoded)

# Apply CSP to all datasets
X_train_csp = []
for trial in preprocessed['train']:
    trial_csp = csp.transform(trial[np.newaxis, :, :])
    X_train_csp.append(trial_csp[0])
X_train_csp = np.array(X_train_csp)

X_val_csp = []
for trial in preprocessed['val']:
    trial_csp = csp.transform(trial[np.newaxis, :, :])
    X_val_csp.append(trial_csp[0])
X_val_csp = np.array(X_val_csp)

X_test_csp = []
for trial in preprocessed['test']:
    trial_csp = csp.transform(trial[np.newaxis, :, :])
    X_test_csp.append(trial_csp[0])
X_test_csp = np.array(X_test_csp)

# =============================================================================
# Step 3: Prepare Data for EEGNet
# =============================================================================
# Reshape data for EEGNet (samples, channels, timepoints, 1)
X_train_eegnet = X_train_csp[..., np.newaxis]
X_val_eegnet = X_val_csp[..., np.newaxis]
X_test_eegnet = X_test_csp[..., np.newaxis]

# Convert labels to categorical
y_train_cat = to_categorical(le.transform(labels['train']))
y_val_cat = to_categorical(le.transform(labels['val']))

print("\nData shapes after preprocessing:")
print(f"Train: {X_train_eegnet.shape}, Labels: {y_train_cat.shape}")
print(f"Validation: {X_val_eegnet.shape}, Labels: {y_val_cat.shape}")
print(f"Test: {X_test_eegnet.shape}")

# =============================================================================
# Step 4: Build and Train EEGNet Model
# =============================================================================
print("\nBuilding EEGNet model...")
model = EEGNet(
    nb_classes=2,
    Chans=CSP_COMPONENTS,
    Samples=MI_SAMPLES
)

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True)
]

print("Training model...")
history = model.fit(
    X_train_eegnet, y_train_cat,
    validation_data=(X_val_eegnet, y_val_cat),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=2
)

# =============================================================================
# Step 5: Evaluate and Generate Predictions
# =============================================================================
# Load best model
model.load_weights('best_model.keras')

# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val_eegnet, y_val_cat, verbose=0)
print(f"\nValidation Accuracy: {val_acc:.4f}, Validation Loss: {val_loss:.4f}")

# Generate predictions for test set
test_preds = model.predict(X_test_eegnet)
test_labels = le.inverse_transform(np.argmax(test_preds, axis=1))

# Create submission file
submission = pd.DataFrame({
    'id': ids['test'],
    'label': test_labels
})
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created: submission.csv")

# =============================================================================
# Visualization
# =============================================================================
# Plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

# Plot CSP patterns
csp_patterns = csp.patterns_
plt.figure(figsize=(10, 6))
for i in range(CSP_COMPONENTS):
    plt.subplot(2, 2, i+1)
    plt.title(f'CSP Pattern {i+1}')
    mne.viz.plot_topomap(
        csp_patterns[i], 
        mne.create_info(EEG_CHANNELS, SFREQ, 'eeg'),
        show=False
    )
plt.tight_layout()
plt.savefig('csp_patterns.png')
plt.show()

2025-07-14 22:02:19.353197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752530539.576213      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752530539.639811      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Starting preprocessing pipeline...

Processing train set (2400 trials)
Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 1.1s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- 

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 0.5s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 0.1s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

  Kept 10 trials after preprocessing

Applying bandpass filtering (8-30 Hz) and CSP...


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Computing rank from data with rank=None
    Using tolerance 87 (2.2e-16 eps * 2250 dim * 1.7e+14  max singular value)
    Estimated rank (data): 2248
    data: rank 2248 computed from 2250 data channels with 0 projectors
    Setting small data eigenvalues to zero (without PCA)
Reducing data rank from 2250 -> 2248
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
